### Load HR features

Read HR features in *Sorted_feature* folder

In [ ]:
import os
import pandas as pd

def read_data(method,second, ox):
    df_raw=[]
    ff_list=[]
    toolnames=["Deepfake","DeepFakeDetection","Face2Face","NeuralTextures","FaceShifter"]
    i=0
    basepath = './Sorted_feature/'
    with os.scandir(basepath) as m_dirs:
        # method
        for m_dir in m_dirs:
            if m_dir.is_dir() and method in m_dir.name:
                next1path=basepath+m_dir.name+'/'
                # second
                with os.scandir(next1path) as dirs:
                    for directory in dirs:
                        if directory.is_dir() and second in directory.name:
                            next2path=next1path+directory.name+'/'
                            with os.scandir(next2path) as directory:
                                # real or fake
                                for real_fake_dir in directory:
                                    if real_fake_dir.is_dir() and real_fake_dir.name == ox:
                                        next3path = next2path+real_fake_dir.name+'/'
                                        real_fake_name=real_fake_dir.name
                                        # final file
                                        with os.scandir(next3path) as real_fake_dir:
                                            # k=[0,0,0]
                                            allsum=0
                                            for filename in real_fake_dir:
                                                finalpath=next3path+filename.name
                                                # fake 
                                                if "fake" in real_fake_name:
                                                    df_raw.append(0) 
                                                    df=pd.read_excel(finalpath, engine='openpyxl') 
                                                    df_raw[i]=df
                                                    i+=1
                                                # real 
                                                else:
                                                    df_raw.append(0)
                                                    df=pd.read_excel(finalpath, engine='openpyxl')
                                                    df_raw[i]=df 
                                                    i+=1

    return df_raw

In [ ]:
method=["CHROM","GREEN","ICA","LGI","PCA", "PBV"]# 
second = ["sec1_div", "sec1_avg", "sec01_div", "sec01_avg", "sec05_avg", "sec05_div"]
ox = ["real", "fake"]

for n in method:
    for i in second:
        for j in ox:
            globals()[n+'_'+j+'_'+i] = read_data(n,i, j)

Check whether the features have been loaded properly

In [ ]:
import numpy as np

for n in method:
  for i in second:
    for j in ox:
      if len(globals()[n+'_'+j+'_'+i])==0:
        print('DataFrame is empty!')
      else:
        print(n+'_'+j+'_'+i+" shape? ",np.shape(globals()[n+'_'+j+'_'+i]))

### T-test

Conduct T-test

In [ ]:
import numpy as np
import statsmodels.stats.api as sms
from scipy.stats import ttest_ind, t
from scipy import stats
import scipy.stats as st
import numpy as np, statsmodels.stats.api as sms

In [ ]:
def set_data(df,region_num,feature_num):
    temp=[]
    for i in range(82):
        temp.append(df[i][region_num][feature_num])
    return temp

In [ ]:
def RMSE(estimates, actual):
    e2 = [(estimate-actual)**2 for estimate in estimates]
    mse = np.mean(e2)
    return np.sqrt(mse)

In [ ]:
def dist(x,y):
    x=np.array((x))
    y=np.array((y))
    eucl=0
    for i in range(len(x)):
        dst=(x[i]-y[i])**2
        eucl+=np.sqrt(dst)
    return eucl

In [ ]:
def ttest_table_RQ0(df_raw, df_fake, method, region_num,feature_num,second,feature,toolname="x"):
    data_raw=[]
    data_fake=[]
    ttest=[]
    ttest.append(method)
    ttest.append(second)
    ttest.append(feature)
    ttest.append(toolname)
    ttest.append(region_num)
    ttest.append(feature_num)
    data_raw=set_data(df_raw, region_num,feature_num)
    data_fake=set_data(df_fake, region_num,feature_num)
    data=data_raw+data_fake
    
    print(data_raw)
    print(data_fake)
    
    levene=stats.levene(data_raw,data_fake)
    ttest.append(levene[0])
    ttest.append(levene[1])

    if levene[1]>0.05:
        ttest.append("y")
        tt=ttest_ind(data_raw,data_fake,equal_var="True")
        ttest.append(tt[0]) 
        ttest.append(tt[1]) 
        MD=np.mean(data_raw)-np.mean(data_fake)
        ttest.append(MD) 
        std1,std2=np.std(data_raw,ddof=1),np.std(data_fake,ddof=1)
        se1=std1/np.sqrt(len(data_raw))
        se2=std2/np.sqrt(len(data_fake))
        sed=np.sqrt(se1**2.0+se2**2.0)
        ttest.append(sed) 
        ttest.append(np.mean(data_raw)) 
        ttest.append(np.mean(data_fake))
        s1= np.var(data_raw) 
        s2= np.var(data_fake) 
        ttest.append(s1)
        ttest.append(s2)
        n1= len(data_raw) 
        n2= len(data_fake) 
        var1= (s1*s1)/n1
        var2= (s2*s2)/n2
        div=((var1*var1)/(n1-1))+((var2*var2)/(n2-1))
        d_f=(var1+var2)*(var1+var2) 
        d_f=d_f/div
        g_d_f = len(data_raw)+len(data_fake)-2 
        ttest.append(d_f) 
        alpha=0.05
        cm=sms.CompareMeans(sms.DescrStatsW(data_raw), sms.DescrStatsW(data_fake))
        conf=cm.tconfint_diff(usevar='unequal') 
        ttest.append(conf[0])
        ttest.append(conf[1])
        if tt[1]<=conf[0] and tt[1]>=conf[1]: 
            ttest.append("y")
        else:
            ttest.append("n")
        ttest.append(dist(data_raw,data_fake))
        return ttest
    else: 
        ttest.append("n")
        tt=ttest_ind(data_raw,data_fake,equal_var="False")
        ttest.append(tt[0])
        ttest.append(tt[1])
        MD=np.mean(data_raw)-np.mean(data_fake)
        ttest.append(MD)
        std1,std2=np.std(data_raw,ddof=1),np.std(data_fake,ddof=1)
        se1=std1/np.sqrt(len(data_raw))
        se2=std2/np.sqrt(len(data_fake))
        sed=np.sqrt(se1**2.0+se2**2.0)
        ttest.append(sed)
        ttest.append(np.mean(data_raw))
        ttest.append(np.mean(data_fake))
        s1= np.var(data_raw)
        s2= np.var(data_fake)
        ttest.append(s1)
        ttest.append(s2)
        n1= len(data_raw)
        n2= len(data_fake)
        var1= (s1*s1)/n1
        var2= (s2*s2)/n2
        div=((var1*var1)/(n1-1))+((var2*var2)/(n2-1))
        d_f=(var1+var2)*(var1+var2)
        d_f=d_f/div
        g_d_f = len(data_raw)+len(data_fake)-2
        ttest.append(d_f) 
        alpha=0.05
        cm=sms.CompareMeans(sms.DescrStatsW(data_raw), sms.DescrStatsW(data_fake))
        conf=cm.tconfint_diff(usevar='unequal')
        ttest.append(conf[0])
        ttest.append(conf[1])
        if tt[1]<=conf[0] and tt[1]>=conf[1]:
            ttest.append("y")
        else:
            ttest.append("n")
        ttest.append(dist(data_raw,data_fake))
        return ttest

In [ ]:
#RQ 0 - div
second = ["sec1", "sec01",  "sec05"]
method=["CHROM","GREEN","ICA","LGI","PBV","PCA"]
ttest_all=[]
PCA_real_sec1_div
for n in method:
  for i in second:
    for l in range(0,9):
      for k in range(0, 3):
          print("- method: ",n, ", second:",i," feature:",k, "region:", l)
          ttest=ttest_table_RQ0(globals()[n+'_'+'real'+'_'+i+'_'+"div"] , globals()[n+'_'+'fake'+'_'+i+'_'+"div"], n, l, k,i,"div")
          ttest_all.append(ttest)

In [ ]:
len(ttest_all)

In [ ]:
#RQ 0 - avg
second = ["sec1", "sec01",  "sec05"]
feature = ['avg'] #, 'div'
method=["CHROM","GREEN","ICA","LGI","PBV","PCA"]

for n in method:
  for i in second:
    for k in range(0, 3):
      print("- method: ",n, ", second:",i," feature:",k)
      ttest=ttest_table_RQ0(globals()[n+'_'+'real'+'_'+i+'_'+"avg"] , globals()[n+'_'+'fake'+'_'+i+'_'+"avg"], n, 0, k,i, "avg")
      ttest_all.append(ttest)

Save results

In [ ]:
# optimal Combination result 
import pandas as pd
df=pd.DataFrame(ttest_all,columns=["method","sec", "feature", 'toolname','region_num',	'feature_num', "f","Sig.","Homogeneity of variance","t","Sig.(2-tailed)","Mean Difference","Std Error Difference","real_mean","fake_mean","real_var","fake_var","df","95% Conf(Lower)","95% Conf(Upper)","inConf","Euclidean"])
df_ttest=df.drop(['toolname'],axis='columns')
result_df=df_ttest[df_ttest['Sig.(2-tailed)']<0.05]
result_df.to_csv('./t-test_result/optimal_combination_result.csv') 
result_df

In [ ]:
# facial region result
rq1_ttest=df_ttest[df_ttest['method']=="GREEN"]
rq1_ttest=rq1_ttest[rq1_ttest['sec']=="sec01"]
rq1_ttest=rq1_ttest[rq1_ttest['feature_num']==0]
rq1_ttest=rq1_ttest.sort_values(by=['t'], axis=0, ascending=False)
rq1_ttest

In [ ]:
writer = pd.ExcelWriter('./t-test_result/facial_region_result.xlsx', engine='xlsxwriter')
rq1_ttest.to_excel(writer, sheet_name = 't-test')
writer.save()

In [ ]:
# extraction interval result
rq2_ttest=df_ttest[df_ttest['method']=="GREEN"]
rq2_ttest=rq2_ttest[rq2_ttest['feature']=='div']
rq2_ttest=rq2_ttest[rq2_ttest['region_num']==4]
rq2_ttest=rq2_ttest[rq2_ttest['feature_num']==0]
rq2_ttest=rq2_ttest.sort_values(by=['t'], axis=0, ascending=False)
rq2_ttest

In [ ]:
rq2_writer = pd.ExcelWriter('./t-test_result/extraction_interval_result.xlsx', engine='xlsxwriter')
rq2_ttest.to_excel(rq2_writer, sheet_name = 't-test')
rq2_writer.save()

In [ ]:
# rPPG extraction method result
rq3_ttest=df_ttest[df_ttest['sec']=="sec01"]
rq3_ttest=rq3_ttest[rq3_ttest['feature']=="div"]
rq3_ttest=rq3_ttest[rq3_ttest['region_num']==4]
rq3_ttest=rq3_ttest[rq3_ttest['feature_num']==0]
rq3_ttest=rq3_ttest.sort_values(by=['t'], axis=0, ascending=False)
rq3_ttest

In [ ]:
rq3_writer = pd.ExcelWriter('./t-test_result/rPPG_extraction_method_result.xlsx', engine='xlsxwriter')
rq3_ttest.to_excel(rq3_writer, sheet_name = 't-test')
rq3_writer.save()

In [ ]:
# feature engineering method result
rq4_ttest=df_ttest[df_ttest['sec']=='sec01']
rq4_ttest=rq4_ttest[rq4_ttest['feature']=="div"]
rq4_ttest=rq4_ttest[rq4_ttest['region_num']==4]
rq4_ttest=rq4_ttest[rq4_ttest['method']=="GREEN"]
rq4_ttest=rq4_ttest.sort_values(by=['t'], axis=0, ascending=False)
rq4_ttest

In [ ]:
writer = pd.ExcelWriter('./t-test_result/feature_engineering_method_result.xlsx', engine='xlsxwriter')
rq4_ttest.to_excel(writer, sheet_name = 't-test')
writer.save()